In [1]:
!pip install pymssql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 7.5 MB/s 


In [5]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from joblib import dump, load

In [6]:

server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" # paste password between quote marks
database = "ghz"

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

In [8]:

data = pd.read_sql(
    """
    select ticker, date, ret, roeq, mom12m
    from data
    where date>='2021-01' and date<='2021-12'
    """, 
    conn
)
data = data.dropna().copy()
data['rnk'] = data.ret.rank(pct=True)

In [17]:
data2 = pd.read_sql(
    """
    select ticker, date, ret, roeq, mom12m
    from data
    where date='2022-01'
    """, 
    conn
)
data2 = data2.dropna().copy()
data2['rnk'] = data2.ret.rank(pct=True)

In [19]:
Xnew = data2[["roeq", "mom12m"]]
ynew = data2["rnk"]
data2["predict"] = model.predict(Xnew)

In [21]:
data2=data2.sort_values(by="predict")
data2.head()

,ticker,date,ret,roeq,mom12m,rnk,predict
1903,SAVA,2022-01,0.012586,-0.018226,6.843108,0.777917,0.382634
1431,STCN,2022-01,-0.200000,-0.201136,1.007624,0.102468,0.389717
974,LEE,2022-01,0.059641,-0.217982,0.969048,0.902969,0.390026
903,BRN,2022-01,-0.068729,-2.057875,0.976378,0.456294,0.391813
277,ORMP,2022-01,-0.372549,-0.105315,3.533019,0.012965,0.391896


In [22]:
data2.iloc[:100].ret.mean()

-0.1252562197658699

In [23]:
data2.iloc[-100:].ret.mean()

-0.03619473542319609

In [14]:

X = data[["roeq", "mom12m"]]
y = data["rnk"]

model = RandomForestRegressor(
  max_depth=4,
  random_state=0
)
model.fit(X,y)

RandomForestRegressor(max_depth=4, random_state=0)

In [15]:
model.score(X,y)

0.024220226375491483

In [ ]:
model.feature_importances_

In [ ]:
x = np.array([.1, .4]).reshape(1,2)
model.predict(x)

In [ ]:
dump(model, "forest1.joblib")

In [ ]:
model = load("forest1.joblib")